# SatDrag Database 

Simple set of code to create a database for the Satellite Drag data.

Read in the csv file and output an HDF5 file which will allow for faster reading as while as searching of the file before reading and writing. 

Also add a column to the data to identify storm times. 



In [1]:
#plot matplotlib figures in the notebook
%matplotlib inline

#print all output in a cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd

In [2]:
# read in the sat drag file set

drop = ['year', 'month', 'day', 'hour', 'minute', 'second']

data_file = 'D:\\data\\VL_sdrag\\combined_data_all_reduced_omni.csv'

sd_dat = pd.read_csv(data_file, parse_dates=[0], infer_datetime_format=True)
sd_dat = sd_dat.drop(columns=drop)


In [3]:
# get storm times

# read in storm start and end times
storm_txt = 'C:\\Users\\murph\\GitHub\\SatDrag\\data\\storms_drag_epochs_no_overlap.txt'
storm_time = pd.read_csv(storm_txt, header=None, skiprows=1, 
                     delim_whitespace=1, names = ['t_st','t_dst','t_en'], parse_dates=[0, 1, 2],
                     infer_datetime_format=True)

storm_time.dtypes

t_st     datetime64[ns]
t_dst    datetime64[ns]
t_en     datetime64[ns]
dtype: object

In [10]:
# loop through the storms and set
# flags in the satellite drag database
# for the different times

sd_dat['storm'] = -1
sd_dat['storm phase'] = -1

for index, row in storm_time.iterrows():
    stp = (sd_dat['Datetime']>=row['t_st']) & (sd_dat['Datetime']<row['t_en'])
    mpp = (sd_dat['Datetime']>=row['t_st']) & (sd_dat['Datetime']<row['t_dst'])
    rpp = (sd_dat['Datetime']>=row['t_dst']) & (sd_dat['Datetime']<row['t_en'])

    sd_dat.loc[stp,'storm'] = 1
    sd_dat.loc[mpp,'storm phase'] = 1
    sd_dat.loc[rpp,'storm phase'] = 2



In [11]:
storm_time['t_st'][247]
storm_time['t_en'][247]
stp = (sd_dat['Datetime']>=storm_time['t_st'][247]) & (sd_dat['Datetime']<storm_time['t_en'][247])
sd_dat.loc[stp,'storm']
sd_dat.loc[stp,'storm phase']


Timestamp('2008-09-01 15:00:00')

Timestamp('2008-09-12 02:00:00')

659382    1
659383    1
659384    1
659385    1
659386    1
         ..
674377    1
674378    1
674379    1
674380    1
674381    1
Name: storm, Length: 15000, dtype: int64

659382    1
659383    1
659384    1
659385    1
659386    1
         ..
674377    2
674378    2
674379    2
674380    2
674381    2
Name: storm phase, Length: 15000, dtype: int64

In [12]:
sd_dat.to_hdf('D:\\data\\VL_sdrag\\satdrag_database.hdf5', key='sd_dat', mode='w', format='table', data_columns='Datetime', complevel=9, complib='blosc:snappy')

In [13]:
import pandas as pd
bb = pd.read_hdf('D:\\data\\VL_sdrag\\satdrag_database.hdf5')
bb.to_hdf('D:\\data\\VL_sdrag\\satdrag_database_zlib.hdf5', key='sd_dat', mode='w', format='table', data_columns='Datetime', complevel=9, complib='blosc:zlib')


In [15]:
sd_dat.to_csv('D:\\data\\VL_sdrag\\satdrag_database.csv')

In [16]:
sd_dat.to_csv('D:\\data\\VL_sdrag\\satdrag_database_times.csv', columns=['Datetime','storm','storm phase'])

In [2]:
bb = pd.read_hdf('D:\\data\\VL_sdrag\\satdrag_database_zlib.hdf5')

In [3]:
bb.head()

,Datetime,SYM/H_INDEX_nT,1-M_AE_nT,Sec,CLat,SLat,SLon,Height,STime,DipLat,...,3-H_KP*10_,3-H_AP_nT,SOLAR_LYMAN-ALPHA_W/m^2,mg_index (core to wing ratio (unitless)),wavelength (nm),irradiance (W/m^2/nm),uncertainty (unitless),Total mass density,storm,storm phase
0,2004-02-01 00:00:00,-3.0,42.0,86392.5,6.0,7.32789,-46.97031,465.480,20.8630,9.49385,...,23.0,9.0,0.00697,0.26771,121.45,0.005673,0.050021,2.382877e-10,1,1
1,2004-02-01 00:01:00,-2.0,92.0,52.5,3.0,3.46651,-47.15423,466.079,20.8674,5.99292,...,23.0,9.0,0.00697,0.26771,121.45,0.005673,0.050021,2.383206e-10,1,1
2,2004-02-01 00:02:00,-2.0,130.0,115.0,0.0,-0.55515,-47.34640,466.939,20.8719,2.44186,...,23.0,9.0,0.00697,0.26771,121.45,0.005673,0.050021,2.383504e-10,1,1
3,2004-02-01 00:03:00,-3.0,85.0,142.5,-3.0,-2.32433,-47.43097,467.392,20.8739,0.90719,...,23.0,9.0,0.00697,0.26771,121.45,0.005673,0.050021,2.383803e-10,1,1
4,2004-02-01 00:04:00,-3.0,75.0,232.5,-9.0,-8.11216,-47.70693,469.193,20.8805,-4.01442,...,23.0,9.0,0.00697,0.26771,121.45,0.005673,0.050021,2.384108e-10,1,1
